In [ ]:
from tqdm.auto import tqdm
import math
from decimal import Decimal
import time
import numpy as np

In [2]:
def rho_union(*args: list[tuple[int, int]], enable_tqdm: bool = False) -> list[tuple[int,int]]:
  union_list = [*args[0]]
  progress_bar = tqdm(desc="list", total=len(args[1:]), disable=not enable_tqdm)
  for rho_right in args[1:]:
    for pair in rho_right:
      base, exp = pair
      indices = [i for i,x in enumerate(union_list) if base == x[0]]
      index = indices[0] if indices else None
      if index is not None:
        union_list[index] = (union_list[index][0], union_list[index][1] + exp)
      else:
        union_list.append(pair)
    progress_bar.update()
  progress_bar.close()
  return union_list

In [3]:
# import big_o
# best, others = big_o.big_o(lambda n: rho_base(n, dynamic=False), big_o.datagen.n_, min_n=10**6, max_n=10**10, n_measures=5000, classes=[big_o.complexities.Logarithmic, big_o.complexities.Polynomial, big_o.complexities.Exponential])
# print(big_o.reports.big_o_report(best, others))

In [7]:
0.95**95

0.007651428115381815

In [ ]:
from rho.rho_base import RhoBase
rho_obj = RhoBase(enable_tqdm=True, trace=False)
rho_obj.rho_rational_number(0.95**95)


In [ ]:
range_bar = tqdm(range(101))
for i in range_bar:
    num = round(0.01 * i, 2)
    res, p = rho_obj.rho_rational_number(num)
    print(num, res)
    range_bar.update()